In [ ]:
# WEBSITE LINK GIVEN TO SCRAP(HOLLAND & KNIGHT)
url = https://www.hklaw.com/en/professionals?practice=3143ab35-d1e0-4345-bcf9-a5cb09120539

In [13]:
                             #***************** DATA SCRAPPING *****************
import pandas as pd
import selenium
from selenium import webdriver
import time

driver = webdriver.Chrome(r"C:\Users\Merit\Downloads\chromedriver_win32 (5)\chromedriver.exe")
driver.maximize_window()

#*****WEBSITE LINK*****
driver.get("https://www.hklaw.com/en/professionals?practice=3143ab35-d1e0-4345-bcf9-a5cb09120539")

name = []
role = []
mail = []
lst = []
urls = []

for i in range(1,16):
        
    #*****GETTING NAME*****
    people_names = driver.find_elements_by_xpath("//a[contains(@class,'people-card__name')]")
    for n in people_names:
        name.append(n.text)
    
    #*****GETTING EMAIL*****
    people_mail = driver.find_elements_by_xpath("//a[@class='people-card__name']//following::a[1]")
    for m in people_mail:
        if "@" in m.text:
                mail.append(m.text)
        else:
            mail.append("-")
    
    #*****GETTING ROLE*****   
    people_role = driver.find_elements_by_xpath("//div[@class='people-card__title']")
    for r in people_role:
        role.append(r.text)
        
    #*****GETTING CITY & PHONE*****  
    city_phone = driver.find_elements_by_xpath("//div[contains(@class,'people-card__cities')]")
    for i in city_phone:
        lst.append(i.text)
        
        #*****GETTING PAGE URL***** 
        page_url = driver.current_url
        urls.append(page_url)
        
    #*****PAGINATION*****
    driver.find_element_by_xpath("//button[@class ='pagination__arrow pagination__next']").click()
    time.sleep(3)

l = []
for i in lst:
    l.append(i.split('\n'))

#*****GETTING CITY*****
city=[]
for i in range(len(l)):
    city.append(l[i][0])
    
#*****GETTING PHONE*****
phone=[]
for i in range(len(l)):
    phone.append(l[i][1])

driver.close()    

details = zip(name,role,mail,city,phone,urls)
detailsDF = pd.DataFrame(details,columns = ['NAME','ROLE','EMAIL','CITY','PHONE','PAGE URLS'])
detailsDF['SNO'] = detailsDF.index + 1
detailsDF.set_index('SNO',inplace=True)

In [14]:
detailsDF

,NAME,ROLE,EMAIL,CITY,PHONE,PAGE URLS
SNO,,,,,,
1,Glenn A. Adams,Partner,Glenn.Adams@hklaw.com,Orlando,407.244.5222,https://www.hklaw.com/en/professionals?practic...
2,Anebi Xavier Adoga,Associate,Anebi.Adoga@hklaw.com,"Washington, D.C.",202.469.5215,https://www.hklaw.com/en/professionals?practic...
3,Paul M. Aguggia,Partner,Paul.Aguggia@hklaw.com,New York - West 52nd St.,212.513.3352,https://www.hklaw.com/en/professionals?practic...
4,Sofia Alcantara,Associate,Sofia.Alcantara@hklaw.com,Mexico City - Reforma,+52.55.3602.8076,https://www.hklaw.com/en/professionals?practic...
5,David B. Allen,Partner,David.Allen@hklaw.com,Dallas - One Arts Plaza,214.964.9437,https://www.hklaw.com/en/professionals?practic...
...,...,...,...,...,...,...
296,Margarita Solorza,Associate,Margarita.Solorza@hklaw.com,Bogotá,+57.601.745.5847,https://www.hklaw.com/en/professionals?practic...
297,Andrew J. Soven,Partner,Andrew.Soven@hklaw.com,Philadelphia,215.252.9554,https://www.hklaw.com/en/professionals?practic...
298,Joseph Speight,Associate,Joseph.Speight@hklaw.com,Charlotte,980.215.7824,https://www.hklaw.com/en/professionals?practic...


In [ ]:
                            #***************** FLASK API & SQLITE *****************
import os
import sqlite3
from flask import jsonify
from flask_api import FlaskAPI
from flask_restful import Api
from sqlalchemy import create_engine
from flask import request
import json
app=FlaskAPI(__name__)
api=Api(app)

a=create_engine('sqlite:///PROJECT_HKLAW.db')

#*****CREATE ROUTE*****     
@app.route('/hklaw_create', methods=['GET'])
def create_table():
    try:
        con = a.connect()
        trans = con.begin()
        con.execute('CREATE TABLE IF NOT EXISTS hklaw_tbl(SNO INT PRIMARY KEY,NAME TEXT,ROLE TEXT,EMAIL TEXT,URLS TEXT)')
        trans.commit()
        message = {"Message " : "Table Created successfully"}
        return message
    except Exception as error:
        message = {"Message " : "Table Creation Failed"}
        return message
    finally:
        con.close()
        
    
#*****INSERT ROUTE***** 
@app.route('/hklaw_insert', methods=['GET'])
def insert_data():
    con = a.connect()
    detailsDF.to_sql('hklaw_tbl', con,if_exists='replace')
    message = {"Message " : "Record Inserted Successfully"}
    return message

        
#*****SELECT ROUTE***** 
@app.route('/hklaw_select', methods=['GET'])
def select_data():
    con = a.connect()
    x = con.execute("select * from hklaw_tbl")  
    return json.dumps([dict(v) for v in x])
app.run()

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [22/Nov/2021 11:34:15] "GET / HTTP/1.1" 404 -
127.0.0.1 - - [22/Nov/2021 11:34:30] "GET /hklaw_create HTTP/1.1" 200 -
127.0.0.1 - - [22/Nov/2021 11:34:30] "GET /flask-api/static/css/bootstrap.min.css HTTP/1.1" 304 -
127.0.0.1 - - [22/Nov/2021 11:34:30] "GET /flask-api/static/css/bootstrap-tweaks.css HTTP/1.1" 304 -
127.0.0.1 - - [22/Nov/2021 11:34:30] "GET /flask-api/static/css/prettify.css HTTP/1.1" 304 -
127.0.0.1 - - [22/Nov/2021 11:34:30] "GET /flask-api/static/js/jquery-1.8.1-min.js HTTP/1.1" 304 -
127.0.0.1 - - [22/Nov/2021 11:34:30] "GET /flask-api/static/css/default.css HTTP/1.1" 304 -
127.0.0.1 - - [22/Nov/2021 11:34:30] "GET /flask-api/static/js/bootstrap.min.js HTTP/1.1" 304 -
127.0.0.1 - - [22/Nov/2021 11:34:30] "GET /flask-api/static/js/default.js HTTP/1.1" 304 -
127.0.0.1 - - [22/Nov/2021 11:34:30] "GET /flask-api/static/js/prettify-min.js HTTP/1.1" 304 -
127.0.0.1 - - [22/Nov/2021 11:34:30] "GET /f